# Identifier les interventions mobilisant l'idée de "République"

Ce notebook vise à présenter le stratégie de recherche pour circonscrire l'analyse de discours aux prises de paroles politiques mobilisant l'idée de "République". 

## Définition préalable de ce concept : 

Énoncés sur la "République" captés par le concept "d'idée politique" au sens d'effectuer un travail sur un ensemble idéel de références organisées à la "République". 

***Nécessité d'écrire une véritable définition !!!!!***

    ==> Statut dual de l'idée de "République" :
        - (1) Mobilisations de références directes à la "République" via l'utilisation de la famille du mot "République" (« république », "républicain", etc.),
        - (2) Mobilisation de références indirectes et implicites à la "République" via son champ lexical. Peut alors comprendre des figures, évènements et valeurs/principes dont la définition/liste n'est pas consensuelle dans la littérature.

## Stratégie de recherche : 

1. Définition d'une regex large inclusive sur la famille de mot "république" [V]
2. Entrainer un classifieur pour exclure les termes de la famille de mot "république" ne renvoyant pas à cette idée
3. Comparer le classifieur à une regex exclusive
    - Améliorer la regex exclusive et la simplifier 
4. Définition d'une regex large pour comprendre tout le champ lexical possible de la "république"
5. Trier les références à l'IR des références extérieures ou indépendantes de l'IR
    - Vérifier le nombre de co-occurences entre "République" (1), les éventuelles références (2), et les co-occurrences entre valeurs et figures, ou valeurs et évènements pour mettre l'accent sur ce qu'il faudrait potentiellement exclure de l'idée de République = essayer d’objectiver si il existe un champ lexical +/- cohérent qui se détache (et ce qui serait étranger/extérieur)  
    - Exclure par annotation manuelle (sur active tigger) les valeurs, figures et évènements qui en fonction de l'auteur, de la date et du contexte linguistique renvoie à une autre idée ou idéologie politique celle de la République

In [1]:
# Ouvrir le fichier propre 
import pandas as pd
import re
import csv

df = pd.read_csv(
    "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_CSS_clean.csv", low_memory=False, dtype={"ID_orateur": str}
)
df.shape

(683680, 26)

## Étape 1 : Regex large famille du mot "République"

Comment définir largement la famille du mot "République" ? En revenant à sa "racine" puisque la famille d'un mot correspond à l'ensemble des mots dérivés de la même racine (ou base/radical) ou d'une racine proche (mer et mar par exemple). Ces mots prennent des formes différentes (verbe, adverbe, nom, adjectif, etc.) en changeant de suffixe, pré-fixe, etc.

Définition théorique : "*on donne le nom de « racine » à cette partie du lexème qui constitue la limite de la segmentation, à la fois porteuse de l’identité du lexème (cette partie d’interprétation qui le différencie de tous les autres lexèmes), et qui est insécable sous peine que soit perdue cette identité lexicale.*" Huot, H. (2006). Chapitre III. Racines et radicaux. Cursus, 2, 41-52.

Ici la base de "République" est "républi" : exemple (républicain, républicanisme, )

In [2]:
# Regex de la famille du mot "République" (simplifié ici) --> passe de 683680 à 37979.

pattern_lexical = re.compile(
    r"républi",
    re.I,
)

def famille_de_mot(texte_propre):
    for match in pattern_lexical.finditer(texte_propre):
        return True
        return False

In [3]:
# Appliquer sur la colonne
df["texte_propre"] = df["texte_propre"].fillna("") # nécessaire de remplacer les 35 NaN restantes par des chaînes vides pour faire tourner la fonction re
df["République_FDM_IN"] = df["texte_propre"].apply(famille_de_mot)

In [4]:
df_répu_in = df[df["République_FDM_IN"] == True]

In [5]:
df_répu_in.shape

(37979, 27)

In [6]:
import csv  

df_répu_in.to_csv(
    "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_CSS_République_inclusive.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)

## Étape 2/3 : Constitution d'une regex exclusive 

De nombres termes appartiennent à la famille du mot "république" sans renvoyer à l'idée de "République" telle que définit ci-dessus, comme :

        * Représentant : “Président de la République” (pour dire le nom de la personne qui l’occupe)
        * Institutions :  “Cour de justice de la république”/"Cour de sûreté de la République"/"procureur de la République"/"Administration générale de la république"
        * Partis ou groupes politiques : "L(l)es Républicains"/"La République en Marche"/"groupe socialistes, écologistes et républicains" (Socialiste, Écologiste et Républicain)/"Gauche démocrate et républicaine"
        * Pays (liste extensive créée pour une regex spécifique)

==> Écrire une regex pour les exclure automatiquement. 

Attention : comme les termes exclus peuvent apparaitre aussi avec les termes voulu, éviter de chainer et finir par virer des trucs qu'on aurait voulu (ex : "les idées républicaines sont menacées par Les Républicains")

In [ ]:
# Regex pays insuffisante car doit aussi comprendre la forme adjectivable des pays et république en minuscule 

# préparer les pays à exclure en utilisant la liste faite par le notebook pays_republique.ipynb
with open("/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/liste_pays_republique_sup.txt", "r", encoding="utf-8") as f:
    liste_pays = [line.strip() for line in f]

# créer un pattern regex pour les pays

# ici pas besoin d'avoir un groupe de capture pas pays mais juste global ok
# pattern_pays = r"(\b(?:" + r"|".join(re.escape(p) for p in liste_pays) + r")\b)"
pattern_pays = r"(?<!\w)(?:" + r"|".join(re.escape(p) for p in liste_pays) + r")(?!\w)" #fonction sans le \b car le b ne reconnait pas la bonne frontière de mot et laisse par exemple République d'Arménie

# = résultat en enlève 5 540 (de 37 979 regex inclusion, à 32 439 regex exclu uniquement pays_compile)


In [ ]:
# Regex des expressions à exclure

# Expressions à exclure - casse exacte 
pattern_excl_case_sensitive = re.compile(
    r"\b[LlDd]es Républicains\b" # (5 378 occurrences = 37 979 - 32 601 ? 18 555 = 32601 - 14046)
)  

# Expressions à exclure - ignorer la casse
pattern_excl_case_insensitive = re.compile(
    r"|(\b[Rr]épublique en marche\b)" #(3 547 occurrences = 17593 - 140046)
    r"|(\b[Pp]résident[s]? de la République\b)" #(13 377 occurences = 27 423- 14 046) 
    r"|(\bgauche démocrate et républicaine\b)" # (921 occurrences = 14 967 - 14046)
    r"|(\bSocialiste, Écologiste et Républicain\b)" # (1 occurrence = 14 065 - 14 046)
    r"|(\bprocureur[s]? de la République\b)" # (844 occurrences = 14890 - 14 046)
    r"|(\bcour[s]? de justice de la République\b)" # (89 occurrences= 14 135 - 14 046)
    r"|(\bcour[s]? de sûreté de la République\b)" # (11 occurrences = 14 057 - 14 046)
    r"|(\badministration générale de la République\b)" # (73 occurrences = 14119 - 14 046)
    r"|(" + pattern_pays + ")"  # ajout des exclusions de pays 5602 occurrences 5540 (sans le problème des 2 types d'apostrophes)
    r"|(\bGouvernement de la République française\b)" # 47 occurrences mais Pose question de si on met ou pas car surtout mobilisé lors de discussion sur la signature d'accords commerciaux entre deux pays (=signifie le gouvernement français + que la république comme idée politique)
    r"|(\brépublique[s] soviétique[s]\b)", # 6 occurrences 
    re.I,
)


def contains_lexical_outside_excl(text):
    # Trouver les positions des expressions exclues
    excl_positions = []

    # Ajouter les exclusions sensibles à la casse
    excl_positions.extend(
        [m.span() for m in pattern_excl_case_sensitive.finditer(text)]
    )

    # Ajouter les exclusions insensibles à la casse
    excl_positions.extend(
         [m.span() for m in pattern_excl_case_insensitive.finditer(text)]
     )

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences de la famille du mot 
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False

## ATTENTION ERREUR À RÉGLER, POURQUOI ÇA PREND RÉPUBLIQUE D'ARMÉNIE ?????
Ne vient pas des accents ! 

In [30]:
# # bloc d'essai
mon_texte = "République d'Arménie"
contains_lexical_outside_excl(mon_texte)


False

In [34]:
# Appliquer sur la colonne
df["texte_propre"] = df["texte_propre"].fillna("") # nécessaire de remplacer les 35 NaN restantes par des chaînes vides pour faire tourner la fonction re
df["Republique_FDM_Regex"] = df["texte_propre"].apply(contains_lexical_outside_excl)

In [35]:
df_match = df[df["Republique_FDM_Regex"]]
df_match.shape

(13931, 28)

On passe de 37979 (regex inclusion) à 13931 (regex exclusion) (à comparer avec le clasifieur entrainé sur AT)

In [24]:
df.to_csv(
    "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_CSS_République_FDM_proportion.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)

In [14]:
# vérif écriture/lecture ok
df_test = pd.read_csv(".../Republique_FDM_Regex.csv")
df_test.shape

FileNotFoundError: [Errno 2] No such file or directory: '.../Republique_FDM_Regex.csv'

# PAUSE ICI

## Étape 3 : Regex inclusive large du champ lexical de la "République"

### Dictionnaire manuel des familles de mots pouvant appartenir au champ lexical de la "République" 

À compléter


Bibliographie indicative : 

Christin, O.,  Soulié, S.  et Worms, F.  (2023). Les 100 mots de la République. (2e éd.). Presses Universitaires de France. https://shs.cairn.info/les-100-mots-de-la-republique--9782715414075?lang=fr.

Duclert, V., & Prochasson, C. (2002). Dictionnaire critique de la République. Flammarion.

Spitz, J.-F. (2022). La République ? Quelles valeurs ? Essai sur un nouvel intégrisme politique. Gallimard; Cairn.info. https://doi.org/10.3917/gall.spitz.2022.01
